In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [44]:
import pandas as pd

In [2]:
%env PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON

from posydon.popsyn.binarypopulation import BinaryPopulation
from posydon.binary_evol import SimulationProperties
from posydon.binary_evol.flow_chart import flow_chart
from posydon.binary_evol.MESA.step_mesa import CO_HeMS_step, MS_MS_step, CO_HMS_RLO_step
from posydon.binary_evol.DT.step_detached import detached_step
from posydon.binary_evol.CE.step_CEE import StepCEE
from posydon.binary_evol.SN.step_SN import StepSN
from posydon.binary_evol.DT.double_CO import DoubleCO
from posydon.binary_evol.step_end import step_end
from posydon.binary_evol.simulationproperties import TimingHooks, StepNamesHooks

import sys
import numpy as np

env: PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON


In [12]:
sheme = 'IF'

if sheme == 'NN':
    interpolation_method='nearest_neighbour'
    save_initial_conditions=True
    track_interpolation=False
elif sheme == 'IF':
    interpolation_method='linear3c_kNN'
    save_initial_conditions=False
    track_interpolation=False  

sim_kwargs = dict(
    flow = (flow_chart, {}),
    step_HMS_HMS = (MS_MS_step, dict(interpolation_method=interpolation_method,
                                     save_initial_conditions=save_initial_conditions,
                                     track_interpolation=track_interpolation)),
    step_CO_HeMS = (CO_HeMS_step, dict(interpolation_method=interpolation_method,
                                     save_initial_conditions=save_initial_conditions,
                                     track_interpolation=track_interpolation)),
    step_CO_HMS_RLO = (CO_HMS_RLO_step, dict(interpolation_method=interpolation_method,
                                     save_initial_conditions=save_initial_conditions,
                                     track_interpolation=track_interpolation)),
    step_detached = (detached_step, {}),
    step_CE = (StepCEE, dict(core_definition_H_fraction= 0.1,
                            )),
    step_SN = (StepSN, {}), 
    step_dco = (DoubleCO, {}),
    step_end = (step_end, {}),
    extra_hooks = [(TimingHooks, {}),(StepNamesHooks, {})],
    verbose=True
)

sim_prop = SimulationProperties(**sim_kwargs)

kwargs = {'number_of_binaries' : 100,
          'primary_mass_min' : 7,
          'primary_mass_max' : 45,
          'secondary_mass_scheme' : 'flat_mass_ratio',
          'secondary_mass_min': 7,
          'secondary_mass_max': 45,
          'orbital_separation_min': 1,
          'orbital_separation_max': 3e3,
          'eccentricity_scheme':'zero',
          'extra_columns' : ['step_times','step_names'], 
          'only_select_columns' : ['state', 'event', 'time', 'lg_mtransfer_rate',
                                   'orbital_period'],
          #'selection_function' : lambda binary: not(binary in pop.find_failed()),
          'include_S1' : True , 
          'S1_kwargs' : {'only_select_columns' : ['state', 'mass', 
                                  'log_R', 'center_h1','center_he4',
                                  'he_core_mass', 'surface_he4', 'surface_h1',
                                  'lg_mdot'],
                        'scalar_names' : ['natal_kick_array', 'SN_type']},
          'include_S2' : True,
          'S2_kwargs' : {'only_select_columns' : ['state', 'mass', 
                                  'log_R', 'center_h1','center_he4',
                                  'he_core_mass', 'surface_he4', 'surface_h1',
                                  'lg_mdot'],
                        'scalar_names' : ['natal_kick_array', 'SN_type']},
          'star_formation' : 'burst',
          'max_simulation_time' : 13.7e9,
         }

pop = BinaryPopulation(
    population_properties=sim_prop,
#     file_name = 'DBH_CE2_dco.h5',
    **kwargs
)

pop.evolve(breakdown_to_df=False, from_hdf=False, tqdm=True, **kwargs)



100%|██████████| 100/100 [00:34<00:00,  2.91it/s]


In [13]:
pop[2].to_df(**kwargs)[['time','S1_mass','S2_mass','orbital_period', 'S1_log_R', 'S2_log_R' ,'S1_lg_mdot','S2_lg_mdot','step_names','event',
                       'S1_state', 'S2_state', 'S1_surface_he4']]

,time,S1_mass,S2_mass,orbital_period,S1_log_R,S2_log_R,S1_lg_mdot,S2_lg_mdot,step_names,event,S1_state,S2_state,S1_surface_he4
binary_index,,,,,,,,,,,,,
2,0,11.8882,10.5153,26.7122,NaN,NaN,NaN,NaN,initial_cond,ZAMS,H-rich_Core_H_burning,H-rich_Core_H_burning,NaN
2,2.13018e+07,3.87821,10.5214,58.6454,0.391064,0.973487,-5.57007,-7.34257,step_HMS_HMS,CC1,stripped_He_Central_C_depletion,H-rich_Core_H_burning,0.986151
2,2.13018e+07,1.17587,10.5214,NaN,NaN,0.973487,NaN,NaN,step_SN,NaN,NS,H-rich_Core_H_burning,NaN
2,2.13018e+07,1.17587,10.5214,NaN,NaN,0.973487,NaN,NaN,step_end,END,NS,H-rich_Core_H_burning,NaN


In [14]:
pop[0].to_df(**kwargs).columns

Index(['state', 'event', 'time', 'lg_mtransfer_rate', 'orbital_period',
       'step_times', 'step_names', 'S1_state', 'S1_mass', 'S1_log_R',
       'S1_center_h1', 'S1_center_he4', 'S1_he_core_mass', 'S1_surface_he4',
       'S1_surface_h1', 'S1_lg_mdot', 'S2_state', 'S2_mass', 'S2_log_R',
       'S2_center_h1', 'S2_center_he4', 'S2_he_core_mass', 'S2_surface_he4',
       'S2_surface_h1', 'S2_lg_mdot'],
      dtype='object')

In [15]:
%env PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON



env: PATH_TO_POSYDON=/projects/b1119/mpg0328/tutorial/POSYDON


In [16]:
import os
PATH_TO_POSYDON = os.environ.get("PATH_TO_POSYDON")

In [17]:
print(PATH_TO_POSYDON)

/projects/b1119/mpg0328/tutorial/POSYDON


In [18]:
os.environ

environ{'PATH': '/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/software/anaconda3/2018.12/bin:/software/qanalytics/jupyterhub/env/bin',
        'LANG': 'en_US.UTF-8',
        'JUPYTERHUB_API_TOKEN': 'e71f88cbdf544dbe9fe8569431854e3d',
        'JPY_API_TOKEN': 'e71f88cbdf544dbe9fe8569431854e3d',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-mpg0328',
        'JUPYTERHUB_HOST': '',
        'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/mpg0328/oauth_callback',
        'JUPYTERHUB_USER': 'mpg0328',
        'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
        'JUPYTERHUB_BASE_URL': '/',
        'JUPYTERHUB_SERVICE_PREFIX': '/user/mpg0328/',
        'USER': 'mpg0328',
        'HOME': '/home/mpg0328',
        'SHELL': '/bin/bash',
        'JPY_PARENT_PID': '72623',
        'TERM': 'xterm-color',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
        'PATH_TO_POSYDON': '/projects/b11

In [19]:

from posydon.binary_evol.singlestar import SingleStar
from posydon.binary_evol.binarystar import BinaryStar


In [85]:
q_array = np.round(np.arange(0.1, 1.1, 0.1),2)
p_array = np.round(10**(np.arange(1, 3.75, 0.25)),2)
m_donor_array = np.arange(10,60,2.5)

In [87]:
i = 0
m_donor = 30
binary_df_list = []
for q in q_array:
    for p in p_array:

        # primary; m_donor
        kwargs1 = dict(state='H-rich_Core_H_burning',
                   mass=m_donor,
                   metallicity=0.014)
        star_1 = SingleStar(**kwargs1)

        # secondary; m_donor*q
        kwargs2 = dict(state='H-rich_Core_H_burning',
                   mass=m_donor*q,
                   metallicity=0.014)
        star_2 = SingleStar(**kwargs2)


        kwargs3 = {'state' : 'detached',
                   'event' : 'ZAMS',
                   'time' : 0.,
                   'orbital_period' : p,
                   'eccentricity' : 0.}
        binary = BinaryStar(star_1, star_2, properties=sim_prop, **kwargs3)

        # evolve
        binary.evolve()
        
        # to pandas dataframe
        binary_df = binary.to_df()
        keep_columns = ['time', 'state', 'event', 'separation', 'orbital_period', 
                'eccentricity','S1_mass','S2_mass','S1_state','S2_state']
        binary_df = binary_df[keep_columns].copy()
                        
        # making binary ID to help parse datagtframe
        binary_df['ID'] = [i]*len(binary_df)
        binary_df.set_index('ID', inplace=True)
        i += 1
                        
        binary_df_list.append(binary_df)
binary_grid_df = pd.concat(binary_df_list)        

/projects/b1119/mpg0328/tutorial/POSYDON/posydon/interpolation/IF_interpolation.py:470: UserWarning: 1NN interpolation used for 1 binaries out of hull.
  warnings.warn(f"1NN interpolation used for {np.sum(wnan)} binaries out of hull.")
/projects/b1119/mpg0328/tutorial/POSYDON/posydon/interpolation/IF_interpolation.py:470: UserWarning: 1NN interpolation used for 1 binaries out of hull.
  warnings.warn(f"1NN interpolation used for {np.sum(wnan)} binaries out of hull.")
/projects/b1119/mpg0328/tutorial/POSYDON/posydon/interpolation/IF_interpolation.py:470: UserWarning: 1NN interpolation used for 1 binaries out of hull.
  warnings.warn(f"1NN interpolation used for {np.sum(wnan)} binaries out of hull.")
/projects/b1119/mpg0328/tutorial/POSYDON/posydon/interpolation/IF_interpolation.py:470: UserWarning: 1NN interpolation used for 1 binaries out of hull.
  warnings.warn(f"1NN interpolation used for {np.sum(wnan)} binaries out of hull.")
/projects/b1119/mpg0328/tutorial/POSYDON/posydon/interpo

In [84]:
q, p, m_donor

(0.65, 17.78, 30)

In [90]:
binary_grid_df.loc[40]

,time,state,event,separation,orbital_period,eccentricity,S1_mass,S2_mass,S1_state,S2_state
ID,,,,,,,,,,
40,0,detached,ZAMS,NaN,562.34,0,30,12,H-rich_Core_H_burning,H-rich_Core_H_burning
40,6.8616e+06,detached,CC1,820.578,555.697,0,11.6871,12.3412,stripped_He_Central_C_depletion,H-rich_Core_H_burning
40,6.8616e+06,detached,NaN,496.31,264.194,0.781915,11.1867,12.3412,BH,H-rich_Core_H_burning
40,1.92502e+07,RLO2,oRLO2,109.861,27.8446,0,11.1867,11.7864,BH,H-rich_Shell_H_burning
40,2.0514e+07,detached,CC2,290.108,146.718,0,11.1955,4.0363,BH,stripped_He_Central_C_depletion
40,2.0514e+07,disrupted,NaN,NaN,NaN,NaN,11.1955,1.30662,BH,NS
40,2.0514e+07,disrupted,END,NaN,NaN,NaN,11.1955,1.30662,BH,NS


In [91]:
binary_grid_df.to_csv('posydon_grid_mdonor_30.csv')

In [21]:
# this fails if you don't run 

# pop = BinaryPopulation(
#     population_properties=sim_prop,
# #     file_name = 'DBH_CE2_dco.h5',
#     **kwargs
# )
# pop.evolve(breakdown_to_df=False, from_hdf=False, tqdm=True, **kwargs)

binary.evolve()

In [23]:
binary.star_1.state_history

['H-rich_Core_H_burning', 'stripped_He_Central_C_depletion', 'NS', 'NS']

In [58]:
keep_columns = ['state', 'event', 'time', 'separation', 'orbital_period', 
                'eccentricity','S1_mass','S2_mass','S1_state','S2_state']
df_test = df_test[keep_columns].copy()
df_test['ID'] = [1]*len(df_test)
df_test.set_index('ID', inplace=True)


display(df_test)

,state,event,time,separation,orbital_period,eccentricity,S1_mass,S2_mass,S1_state,S2_state
ID,,,,,,,,,,
1,detached,ZAMS,0,NaN,100,0,20,10,H-rich_Core_H_burning,H-rich_Core_H_burning
1,detached,CC1,1.02944e+07,233.096,99.2242,0,7.01927,10.2555,stripped_He_Central_C_depletion,H-rich_Core_H_burning
1,disrupted,NaN,1.02944e+07,NaN,NaN,NaN,1.36319,10.2555,NS,H-rich_Core_H_burning
1,disrupted,END,1.02944e+07,NaN,NaN,NaN,1.36319,10.2555,NS,H-rich_Core_H_burning


In [46]:
pd.concat(test)

,state,event,time,separation,orbital_period,eccentricity,S1_mass,S2_mass,S1_state,S2_state
binary_index,,,,,,,,,,
NaN,detached,ZAMS,0,NaN,100,0,20,10,H-rich_Core_H_burning,H-rich_Core_H_burning
NaN,detached,CC1,1.02944e+07,233.096,99.2242,0,7.01927,10.2555,stripped_He_Central_C_depletion,H-rich_Core_H_burning
NaN,disrupted,NaN,1.02944e+07,NaN,NaN,NaN,1.36319,10.2555,NS,H-rich_Core_H_burning
NaN,disrupted,END,1.02944e+07,NaN,NaN,NaN,1.36319,10.2555,NS,H-rich_Core_H_burning
NaN,detached,ZAMS,0,NaN,100,0,20,10,H-rich_Core_H_burning,H-rich_Core_H_burning
NaN,detached,CC1,1.02944e+07,233.096,99.2242,0,7.01927,10.2555,stripped_He_Central_C_depletion,H-rich_Core_H_burning
NaN,disrupted,NaN,1.02944e+07,NaN,NaN,NaN,1.36319,10.2555,NS,H-rich_Core_H_burning
NaN,disrupted,END,1.02944e+07,NaN,NaN,NaN,1.36319,10.2555,NS,H-rich_Core_H_burning


In [54]:
cols = np.array(df_test.columns.tolist())
print(cols['MT' in cols ])
print(cols)

[]
['state' 'event' 'time' 'separation' 'orbital_period' 'eccentricity'
 'V_sys' 'rl_relative_overflow_1' 'rl_relative_overflow_2'
 'lg_mtransfer_rate' 'mass_transfer_case' 'trap_radius' 'acc_radius'
 't_sync_rad_1' 't_sync_conv_1' 't_sync_rad_2' 't_sync_conv_2'
 'nearest_neighbour_distance' 'p_classification' 'S1_state'
 'S1_metallicity' 'S1_mass' 'S1_log_R' 'S1_log_L' 'S1_lg_mdot'
 'S1_lg_system_mdot' 'S1_lg_wind_mdot' 'S1_he_core_mass'
 'S1_he_core_radius' 'S1_c_core_mass' 'S1_c_core_radius' 'S1_o_core_mass'
 'S1_o_core_radius' 'S1_co_core_mass' 'S1_co_core_radius' 'S1_center_h1'
 'S1_center_he4' 'S1_center_c12' 'S1_center_n14' 'S1_center_o16'
 'S1_surface_h1' 'S1_surface_he4' 'S1_surface_c12' 'S1_surface_n14'
 'S1_surface_o16' 'S1_log_LH' 'S1_log_LHe' 'S1_log_LZ' 'S1_log_Lnuc'
 'S1_c12_c12' 'S1_center_gamma' 'S1_avg_c_in_c_core' 'S1_surf_avg_omega'
 'S1_surf_avg_omega_div_omega_crit' 'S1_total_moment_of_inertia'
 'S1_log_total_angular_momentum' 'S1_spin' 'S1_conv_env_top_mass'
 'S1

In [20]:
df_test['S1_state']

binary_index
NaN              H-rich_Core_H_burning
NaN    stripped_He_Central_C_depletion
NaN                                 NS
NaN                                 NS
Name: S1_state, dtype: object